Colorize black and white images.

In [1]:
import numpy as np
import cv2 as cv

In [5]:
prototxt_path = 'models/colorization_deploy_v2.prototxt'
model_path = 'models/colorization_release_v2.caffemodel'
kernel_path = 'models/pts_in_hull.npy'
image_path = 'tucan.jpg'

In [6]:
# Select desired model
net = cv.dnn.readNetFromCaffe(prototxt_path, model_path)

pts_in_hull = np.load(kernel_path) # load cluster centers

In [7]:
# populate cluster centers as 1x1 convolution kernel
pts_in_hull = pts_in_hull.transpose().reshape(2, 313, 1, 1)
net.getLayer(net.getLayerId('class8_ab')).blobs = [pts_in_hull.astype(np.float32)]
net.getLayer(net.getLayerId('conv8_313_rh')).blobs = [np.full([1, 313], 2.606, np.float32)]

In [8]:
#LAB -> L = lightness a* b*
bw_image = cv.imread(image_path)
normalized = bw_image.astype("float32") / 255.0
lab = cv.cvtColor(normalized, cv.COLOR_BGR2LAB)

#Resize image to network input size
resized = cv.resize(lab, (224, 224)) # resize image to network input size
L = cv.split(resized)[0]
L -= 50 # subtract 50 for mean-centering

In [9]:
net.setInput(cv.dnn.blobFromImage(L))
ab = net.forward()[0,:,:,:].transpose((1,2,0)) # this is our result

ab = cv.resize(ab, (bw_image.shape[1], bw_image.shape[0]))
L = cv.split(lab)[0]

#Combine lightness with colors
colorized = np.concatenate((L[:, :, np.newaxis], ab), axis=2)
colorized = cv.cvtColor(colorized, cv.COLOR_LAB2BGR)
colorized = (255.0 * colorized).astype("uint8")

cv.imshow("BW Image", bw_image)
cv.imshow("Colorized", colorized)
cv.waitKey(0)
cv.destroyAllWindows()
